In [1]:
import os 
import warnings
import time
import numpy as np
import scipy.stats
import sys
import sklearn
import sklearn.datasets

from pyspark.sql import SparkSession
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("feature_selection") \
    .getOrCreate()

sc=spark.sparkContext

22/05/24 19:40:26 WARN Utils: Your hostname, pierre-hp resolves to a loopback address: 127.0.1.1; using 192.168.0.194 instead (on interface eno1)
22/05/24 19:40:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 19:40:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
train_sessions_engineered = spark.read.csv('../dressipi_recsys2022/session_engineered_features.txt',header=False,
                                          inferSchema=True)

train_purchases = spark.read.load('../dressipi_recsys2022/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [3]:
# sort by session
X = train_sessions_engineered.orderBy('_c0') 
X = X.drop('_c0')
X.take(10)


X_np = np.array(X.collect())
t_X = X_np.transpose()
n_total_features = t_X.shape[0]
print(t_X)
# we want the nb partition to be between 2 or 3 times more than the number of core in our computer
Nb_partition=10
X_RDD = sc.parallelize(t_X,Nb_partition)

22/05/24 19:40:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[[3.1200000e+02 0.0000000e+00 1.6300000e+02 ... 0.0000000e+00
  4.3600000e+02 2.5091000e+04]
 [3.0000000e+00 0.0000000e+00 2.0000000e+00 ... 3.0000000e+00
  3.0000000e+00 3.0000000e+00]
 [5.0000000e+00 4.0000000e+00 4.0000000e+00 ... 5.0000000e+00
  2.0000000e+00 4.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.4285715e-01 0.0000000e+00]]


In [4]:

 # get Y output from the training set sorted by session id
train_purchases_without_date = train_purchases.drop('date')

train_purchases_reformated = train_purchases_without_date.withColumnRenamed("item_id", "item_id_purchased")

Y_order_by_session_id = train_purchases_reformated.drop('session_id')



Y_numpy = np.array(Y_order_by_session_id.collect())
Y_numpy = Y_numpy.flatten()

# Feature Selection

After the feature engineering, we could have a lot of features which are not really worth to give to the predicting model. Considering this problem, we want to use some feature selection algorithms to take the feature which are the most interesting to the model.

We were asked to implement two scalable feature selection algorithms, a ranking algorithm and a forward feature selection.

We could be interested in Minimum-redundancy-maximum-relevance (mRMR) feature selection.


In [5]:
def get_score_mrmr(x, y):
    """
    return the correlation value between two variable in absolute value
    """

    return np.abs(scipy.stats.pearsonr(x, y)[0])

In [6]:
def get_mrmr_score_spark(x,selected_features):
    """
    x : the feature to evaluate to add into the model
    y : output value
    selected_features : the features already selected
    return the score for x with the rest of selected variables
    """
    
    y = broadcast_Y.value
    # Get correlation score between feature x and output y (relevance)
    score_x_y_s = get_score_mrmr(x, y)
    
    
    nb_selected_features = selected_features.shape[0]
    # If some features have already been selected
    if nb_selected_features > 0:
        
        # Get corrrelation scores between x and each feature already selected (redundancy)
        score_features_x_s = np.zeros(nb_selected_features, dtype=float)
        
        for j in range(nb_selected_features):
            
            score_x_s_j = get_score_mrmr(x, selected_features[j,:])
            
            # if score is nan considering that we want to calculate the mean
            # we transform it in 0 ?
            if np.isnan(score_x_s_j):
                score_x_s_j=0
            
                
            score_features_x_s[j] = score_x_s_j
        
                
        # Final score is relevance to output feature - average redundancy with already selected features
        score_x_y_s = score_x_y_s - np.mean(score_features_x_s)
        
    return score_x_y_s

In [7]:
def mrmr_spark(n_total_features, K, sc, X_RDD):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution using mrmr
    """
    time_execution = []
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    

    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()
        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        print(selected_features)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `get_mrmr_score_spark` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0],selected_features)).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected
        scores = np.array(scores)
        
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution

In [8]:
broadcast_Y = sc.broadcast(Y_numpy)
selected_features_indices, execution_time = mrmr_spark(n_total_features, 10, sc,X_RDD)

Step: 0


22/05/24 19:41:13 WARN TaskSetManager: Stage 10 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.
22/05/24 19:41:14 WARN TaskSetManager: Stage 11 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.
22/05/24 19:41:15 WARN TaskSetManager: Stage 12 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.


[]


22/05/24 19:41:16 WARN TaskSetManager: Stage 13 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.
/home/pdefraene/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


5.072085618972778
Step: 1


...

5.980298280715942
Step: 9


22/05/24 19:41:59 WARN TaskSetManager: Stage 47 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.
22/05/24 19:42:00 WARN TaskSetManager: Stage 48 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.


[[  312.     0.   163. ...     0.   436. 25091.]
 [ 2020.  2020.  2020. ...  2020.  2020.  2020.]
 [ 9655. 15654. 18316. ... 25357. 15853. 27400.]
 ...
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]
 [    0.     0.     0. ...     0.     0.     0.]]


22/05/24 19:42:01 WARN TaskSetManager: Stage 49 contains a task of very large size (23442 KiB). The maximum recommended task size is 1000 KiB.


6.704560279846191


In [9]:
selected_features_indices

[36, 19, 4, 32, 0, 23, 5, 18, 21, 34]

Then, we will look a the forward feature Selection by training on a decision tree with every feature and add the features which get the best results. We have implemented it with two stop for the algorithm. The first one is a classic stop when we have K features, the second one stop when the model doesn't increase in accuracy anymore. 

In [10]:
from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import train_test_split

def fit_get_score(x,selected_features):
    """
    train an model to get the score with the addition of a specified x feature
    """    
    
    y = broadcast_Y.value
    n_selected_features = selected_features.shape[0]
    
    if n_selected_features>0:
        # need to merge x and the already selected features
        x = np.vstack((selected_features,x))
        x = x.transpose()
    else:
        x = x.reshape(-1, 1)
        
    
    #split data
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.3)
    
    
    
    # train on training data
    model = tree.DecisionTreeClassifier()
    model = model.fit(x_train,y_train)
    
    # get score on testing set
    return model.score(x_test, y_test)

In [11]:
def forward_feature_selection(n_total_features, K, sc, X_RDD):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        print("best_score :", np.max(scores))
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution
    

In [18]:
X_np_split = X_np[:10000]
t_X_split = np.transpose(X_np_split)
n_total_features_split = t_X_split.shape[0]
# we want the nb partition to be between 2 or 3 times more than the number of core in our computer
Nb_partition=10
X_RDD_split = sc.parallelize(t_X_split,Nb_partition)

Y_numpy_split = Y_numpy[:10000]
broadcast_Y = sc.broadcast(Y_numpy_split)

selected_features_indices_forward, execution_time_forward = forward_feature_selection(n_total_features_split, 20, sc,X_RDD_split)
selected_features_indices_forward

Step: 0


(3000, 1)3:>                                                       (0 + 8) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)3:=====>                                                  (1 + 8) / 10]
(3000, 1)
(3000, 1)
(3000, 1)3:================>                                       (3 + 7) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)3:======================>                                 (4 + 6) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)3:============================>                           (5 + 5) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)3:=================================>                      (6 + 4) / 10]
(3000, 1)
(3000, 1)3:============================================>           (8 + 2) / 10]
(3000, 1)3:==================================================>     (9 + 1) / 10]
(3000, 1)


best_score : 0.01633333333333333
6.262602090835571
Step: 1


(3000, 2)7:>                                                       (0 + 8) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)7:=====>                                                  (1 + 8) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)7:===========>                                            (2 + 8) / 10]
(3000, 2)7:======================>                                 (4 + 6) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)7:============================>                           (5 + 5) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)7:=================================>                      (6 + 4) / 10]
(3000, 2)7:=======================================>                (7 + 3) / 10]
(3000, 2)
(3000, 2)7:============================================>           (8 + 2) / 10]
(3000, 2)7:==================================================>     (9 + 1) / 10]
(

best_score : 0.021
17.7170090675354
Step: 2


...

best_score : 0.021666666666666667
15.60000205039978
Step: 19


(3000, 20):=====>                                                  (1 + 8) / 10]
(3000, 20)
(3000, 20)
(3000, 20)
(3000, 20)
(3000, 20)
(3000, 20):======================>                                 (4 + 6) / 10]
(3000, 20)
(3000, 20)
(3000, 20)
(3000, 20):============================>                           (5 + 5) / 10]
(3000, 20)
(3000, 20):=================================>                      (6 + 4) / 10]
(3000, 20)
(3000, 20)
(3000, 20)
(3000, 20):=======================================>                (7 + 3) / 10]
(3000, 20):============================================>           (8 + 2) / 10]


best_score : 0.021333333333333333
14.049736738204956


(3000, 20)


[5, 29, 15, 23, 18, 24, 36, 34, 16, 12, 37, 33, 19, 32, 20, 35, 21, 27, 13, 26]

In [13]:
def forward_feature_selection_diff(n_total_features, inc_accuracy, sc, X_RDD):
    """
    n_total_features : number of total features
    inc_accuracy : increase of accuracy needed to continue the algorithm
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    last_best_score = 0
    diff_accuracy = 1
    k = 0
    while diff_accuracy > inc_accuracy:
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        # compute the difference between last result and new result
        best_score = np.max(scores)
        diff_accuracy = best_score - last_best_score
        
        
        print("best_accuracy:", best_score)
        
        if best_score > last_best_score:
        
            index_max_score_features = np.argmax(scores)
    
            selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
            del(remaining_features_indices[index_max_score_features])
        
        last_best_score = best_score
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
        k += 1
        
    return selected_features_indices, time_execution

In [16]:
selected_features_indices_forward_diff, execution_time_forward_diff = forward_feature_selection_diff(n_total_features, 0.001, sc,X_RDD_split)
selected_features_indices_forward_diff

Step: 0


(3000, 1):>                                                        (0 + 8) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):=================>                                       (3 + 7) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):======================>                                  (4 + 6) / 10]
(3000, 1)
(3000, 1):============================>                            (5 + 5) / 10]
(3000, 1)
(3000, 1)
(3000, 1):==================================>                      (6 + 4) / 10]
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1)
(3000, 1):=======================================>                 (7 + 3) / 10]
(3000, 1):=============================================>           (8 + 2) / 10]
(3000, 1):===================================================>     (9 + 1) / 10]
(3000, 1)


best_accuracy: 0.01633333333333333
6.362556457519531
Step: 1


(3000, 2):>                                                        (0 + 8) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):=====>                                                   (1 + 8) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):===========>                                             (2 + 8) / 10]
(3000, 2)
(3000, 2):======================>                                  (4 + 6) / 10]
(3000, 2)
(3000, 2)
(3000, 2):============================>                            (5 + 5) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):==================================>                      (6 + 4) / 10]
(3000, 2)
(3000, 2)
(3000, 2)
(3000, 2):=======================================>                 (7 + 3) / 10]
(3000, 2)
(3000, 2)
(3000, 2):===================================================>     (9 + 1) / 10]
(3000, 2)


best_accuracy: 0.021
17.79098916053772
Step: 2


(3000, 3):>                                                        (0 + 8) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3):=====>                                                   (1 + 8) / 10]
(3000, 3):===========>                                             (2 + 8) / 10]
(3000, 3)
(3000, 3)
(3000, 3):=================>                                       (3 + 7) / 10]
(3000, 3)
(3000, 3)
(3000, 3):======================>                                  (4 + 6) / 10]
(3000, 3)
(3000, 3):============================>                            (5 + 5) / 10]
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3)
(3000, 3):==================================>                      (6 + 4) / 10]
(3000, 3)
(3000, 3):=======================================>                 (7 + 3) / 10]
(3000, 3)
(3000, 3):===================================================>     (9 + 1) / 10]


best_accuracy: 0.021333333333333333
16.75369882583618


(3000, 3)


[5, 29, 15]

In [15]:
#spark.stop()